In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import oandapyV20
import oandapyV20.endpoints.instruments as instruments
from oandapyV20.exceptions import V20Error

# Define your API key and other necessary parameters
api_key='xFfGAaKeYAvDG3MHXpH2aW1SQ8I1GhnNobiHZ9utbZKSzrQ53JBqtG8ABzZ3dtlO'
account_id='56663734'
account_balance = 2000
risk_per_trade = 0.01  # Risk 1% of account balance per trade
max_trade_percent = 1
higher_time_frame = 'daily'
moving_avg_period = 20
key_levels = [0.700, 0.730, 0.760]  # Define key levels
trades = []  # Store information about executed trades

# Use an API to get historical price data
client = oandapyV20.API(access_token=api_key, environment="practice")
params = {
    "granularity": "M1",
    "count": 5000,
    "from": "2023-03-01T00:00:00.000000Z"
}
instrument = "AUD_USD"
try:
    r = instruments.InstrumentsCandles(instrument=instrument, params=params)
    rv = client.request(r)
except V20Error as e:
    print(f"Error: {e}")

# Parse the API response
data = []
for candle in rv['candles']:
    data.append({
        'Date': candle['time'],
        'Open': candle['mid']['o'],
        'High': candle['mid']['h'],
        'Low': candle['mid']['l'],
        'Close': candle['mid']['c'],
        'Volume': candle['volume']
    })

# Create a DataFrame with the parsed data
df = pd.DataFrame(data)
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

# Draw rectangles or zones around the key levels and plot the moving average
for level in key_levels:
    plt.axhspan(level - 0.001, level + 0.001, alpha=0.2, color='gray')
df['SMA'] = df['Close'].rolling(moving_avg_period).mean()
plt.plot(df['SMA'], color='red')

# Draw rectangles or zones around the key levels and plot the moving average
for level in key_levels:
    plt.axhspan(level - 0.001, level + 0.001, alpha=0.2, color='gray')
df['SMA'] = df['Close'].rolling(moving_avg_period).mean()
plt.plot(df['SMA'], color='red')

# Use the moving average to enter trades on the lower time frame chart
for i in range(moving_avg_period, len(df)):
    trade_size = account_balance * risk_per_trade
    stop_loss = 0
    take_profit = 0
    
for i in range(1, len(df)):
    if df['Close'][i] > df['SMA'][i] and df['Close'][i-1] <= df['SMA'][i-1]:
        # Place a buy trade with a stop loss at the key level below and take profit at the next key level
        entry_price = df['Close'][i]
        stop_loss = min(key_levels, key=lambda x: abs(x - entry_price))
        take_profit = [x for x in key_levels if x > entry_price] + [entry_price * (1 + max_trade_percent)]
        take_profit = min(take_profit)
        trade_size = account_balance * risk_per_trade / stop_loss_distance
        trades.append({'index': i, 'entry_time': df.index[i], 'entry_price': entry_price, 'stop_loss': stop_loss,
                       'take_profit': take_profit, 'trade_size': trade_size, 'risk_per_trade': risk_per_trade})
        print(f"Buy trade placed on {df['Date'][i]} at {entry_price}. Stop loss: {stop_loss}. Take profit: {take_profit}")
        
    elif df['Close'][i] < df['SMA'][i] and df['Close'][i-1] >= df['SMA'][i-1]:
        # Place a sell trade with a stop loss at the key level above and take profit at the next key level
        entry_price = df['Close'][i]
        stop_loss = max(key_levels, key=lambda x: abs(x - entry_price))
        take_profit = [x for x in key_levels if x < entry_price] + [entry_price * (1 - max_trade_percent)]
        take_profit = max(take_profit)
        trade_size = account_balance * risk_per_trade / stop_loss_distance
        trades.append({'index': i, 'entry_time': df.index[i], 'entry_price': entry_price, 'stop_loss': stop_loss,
                       'take_profit': take_profit, 'trade_size': trade_size, 'risk_per_trade': risk_per_trade})
        print(f"Sell trade placed on {df['Date'][i]} at {entry_price}. Stop loss: {stop_loss}. Take profit: {take_profit}")

# Wait for definitive rejections before entering trades on the lower time frame chart based on the key levels identified in step 3
if higher_time_frame == 'daily':
    if df['Close'][i] >= key_levels[0]:
        # Wait for a definitive rejection before entering a buy trade on the lower time frame chart
        while df['Close'][i] >= key_levels[0]:
            i += 1
        i -= 1  # go back one step to the last price before the rejection
    elif df['Close'][i] <= key_levels[1]:
        # Wait for a definitive rejection before entering a sell trade on the lower time frame
        while df['Close'][i] <= key_levels[1]:
            i += 1
        i -= 1  # go back one step to the last price before the rejection
    
    # Determine take profit and stop loss levels based on the proximity of the key levels
for level in key_levels:
    if abs(df['Close'][i] - level) <= 0.001:
        if df['Close'][i] > level:
            take_profit = key_levels[key_levels.index(level)+1]
            stop_loss = level
        else:
            take_profit = key_levels[key_levels.index(level)-1]
            stop_loss = level
   
    # Avoid entering trades at key levels and always be aware of key psychological and bank levels
    if df['Close'][i] >= key_levels[0]:
    
    # Avoid entering a buy trade at key level and wait for a definitive rejection
        while df['Close'][i] >= key_levels[0]:
            i += 1
        i -= 1  # go back one step to the last price before the rejection
    elif df['Close'][i] <= key_levels[1]:
    # Avoid entering a sell trade at key level and wait for a definitive rejection
        while df['Close'][i] <= key_levels[1]:
            i += 1
        i -= 1  # go back one step to the last price before the rejection
    
# Always use proper risk management techniques such as setting stop loss levels and never risking more than a certain percentage of your account balance on a single trade.
if trade_size > 0:
    stop_loss_distance = abs(df['Close'][i] - stop_loss)
    take_profit_distance = abs(df['Close'][i] - take_profit)
    stop_loss_price = df['Close'][i] - stop_loss_distance
    take_profit_price = df['Close'][i] + take_profit_distance
    risk_amount = trade_size * stop_loss_distance
    account_balance -= risk_amount

    if df['Close'][i] > stop_loss_price:
        trade_result = 'LOSS'
        account_balance -= trade_size * stop_loss_distance
    elif df['Close'][i] < take_profit_price:
        trade_result = 'PROFIT'
        account_balance += trade_size * take_profit_distance
    else:
        trade_result = 'BREAKEVEN'
        account_balance -= trade_size * 0.5 * stop_loss_distance

    print(f"Trade executed on {df['Date'][i]} at {df['Close'][i]}. Result: {trade_result}")
    print(f"New account balance: {account_balance}")


# Implement a trailing stop loss to lock in profits and minimize losses in case the price moves against you
if len(trades) > 0 and 'exit_time' not in trades[-1]:
    trade = trades[-1]
    stop_loss = trade['stop_loss']
    trailing_stop_distance = trade['entry_price'] - stop_loss
    trailing_stop_price = trade['entry_price'] - trailing_stop_distance
    for i in range(trade['index'], len(df)):
        if df['Close'][i] > trailing_stop_price:
            trailing_stop_distance = df['Close'][i] - stop_loss
            trailing_stop_price = df['Close'][i] - trailing_stop_distance
        elif df['Close'][i] < stop_loss:
            exit_price = stop_loss
            exit_time = df.index[i]
            trades[-1]['exit_time'] = exit_time
            trades[-1]['exit_price'] = exit_price
            account_balance -= trade['trade_size'] * trade['risk_per_trade']
            print(f"Trailing stop loss executed on {exit_time} at {exit_price}. Result: LOSS")
            print(f"New account balance: {account_balance}")
            break
        elif df['Low'][i] <= trailing_stop_price:
            exit_price = trailing_stop_price
            exit_time = df.index[i]
            trades[-1]['exit_time'] = exit_time
            trades[-1]['exit_price'] = exit_price
            account_balance += trade['trade_size'] * (exit_price - trade['entry_price'])
            print(f"Trailing stop loss executed on {exit_time} at {exit_price}. Result: PARTIAL PROFIT")
            print(f"New account balance: {account_balance}")
            break

# Print summary statistics
print(f"\nNumber of trades executed: {len(trades)}")
if len(trades) > 0:
    win_rate = sum([1 for trade in trades if trade['exit_price'] > trade['entry_price']]) / len(trades)
    profit = sum([trade['exit_price'] - trade['entry_price'] for trade in trades if trade['exit_price'] > trade['entry_price']])
    loss = sum([trade['entry_price'] - trade['exit_price'] for trade in trades if trade['exit_price'] <= trade['entry_price']])
    print(f"Win rate: {win_rate:.2f}")
    print(f"Total profit: {profit:.2f}")
    print(f"Total loss: {loss:.2f}")
    print(f"Net profit: {profit + loss:.2f}")

# Plot results
df['Close'].plot(figsize=(16, 8))
for trade in trades:
    if 'exit_time' in trade:
        plt.plot([trade['entry_time'], trade['exit_time']], [trade['entry_price'], trade['exit_price']], 'ro-', lw=3)
    else:
        plt.plot(trade['entry_time'], trade['entry_price'], 'bo', markersize=10)
plt.title("Backtested Trading Strategy Results")
plt.xlabel("Date")
plt.ylabel("Price")
plt.show()


ImportError: numpy.core.multiarray failed to import